In [1]:
import math
import functools

In [2]:
def list_to_binary(l):
    b = 0
    for i in l:
        b += 1<<i
    return b

def decode(b, n):
    return b.to_bytes(n, 'big').decode('utf8')

def largest_power(b):
    s = bin(b)[2:]
    p = len(s) - 1
    for h in s:
        if h == '1':
            return p
        p -= 1
    return -1 # if there is no largest power, return -1 actually, since it's all 0!! and a 0x01 would be largest power 0 since x^0

# this should be implementable in hardware, combinationally, rather than string manipulation
def largest_power_hardware(a):
    p = 0
    while a > 1:
        a >>= 1
        p += 1
    return p

def difference_power_hardware(a, b):
    """Take ints a and b, where b is known to be smaller. Return and int that is the number of places values 
    each a and b's MSB is different. The difference in degree n of each polynomial representation."""
    original_b = b
    p = 0
    while (a ^ b) > original_b:
        a >>= 1
        p += 1
    return p

def powers(b):
    s = bin(b)[2:]
    p = len(s) - 1
    ps = []
    for h in s:
        if h == '1':
            ps.append(p)
        p -= 1
    return ps

In [3]:
# The following notebook will compute a CRC 16 on a message

# divide the following
# x^38 + x^35 + x^30 + x^29 + x^27 + x^24 + x^21 + x^16  <-- divisor / message + padded zeroes for room for CRC
# by
# x^16 + x^12 + x^5 + 1  <-- generator
# result will be the remainder, which we will subtract from the divisor (aka the MESSAGE), and that will be our 
# full message + CRC

divisor = [38, 35, 30, 29, 27, 24, 21, 16] # this was given before, as the message for 'Hi!'
GENERATOR16 = [16, 12, 5, 0]
GENERATOR32 = 0x104C11DB7

In [4]:
def frame_message(message='Hi!', crc_size=16):
    encoded_message = int.from_bytes(message.encode('utf8'), signed=False)
    encoded_message <<= crc_size # bitshift for crc16
    return encoded_message

assert powers(frame_message()) == divisor

In [5]:
message = 'Abc..is easy as 123'
CRC_SIZE = 32
encoded_message = frame_message(message, crc_size=CRC_SIZE)

# Determine size of crc with generator ...
if CRC_SIZE == 32:
    generator_bin = GENERATOR32
elif CRC_SIZE == 16:
    generator_bin = list_to_binary(GENERATOR16)
else:
    print('invalid CRC size')

In [6]:
message_bin = encoded_message # list_to_binary(divisor)
print(f"Encoded message in divisor polynomial is: \n\"{decode(message_bin, max(divisor))}\"")

Encoded message in divisor polynomial is: 
"               Abc..is easy as 123    "


In [7]:
def crc_factory(generator, verbose=False):
    return functools.partial(crc, generator_bin=generator)

def crc(divisor_bin, generator_bin):
    while abs(generator_bin) < abs(divisor_bin): # largest_power(generator_bin) < largest_power(divisor_bin)
        # find difference in place values (difference in degree of the polynomials) of divisor and generator
        p_diff = difference_power_hardware(divisor_bin, generator_bin) # m_deg - g_deg
        
        # multiply g(x) by the difference in highest powers
        d = generator_bin << p_diff
        
        # do r - d  or XOR r with d, subtract the result to keep going with long division..
        divisor_bin = divisor_bin ^ d
    return divisor_bin

remainder = crc(message_bin, generator_bin)

In [8]:
# 'subtract' the remainder so now our message + crc will give us 0 when we run it
# through the same polynomial division!
message_w_crc = message_bin ^ remainder 
assert 0 == crc(message_w_crc, generator_bin)

In [9]:
print(bin(message_w_crc)[2:])
s = bin(message_w_crc)[2:-CRC_SIZE]
print(s)
verified_message = int(s, base=2).to_bytes(math.ceil(len(s)/8), 'big').decode('utf8')
verified_message

100000101100010011000110010111000101110011010010111001100100000011001010110000101110011011110010010000001100001011100110010000000110001001100100011001111001010100110101010101111111000
1000001011000100110001100101110001011100110100101110011001000000110010101100001011100110111100100100000011000010111001100100000001100010011001000110011


'Abc..is easy as 123'

In [10]:
# CRC PER 802.3 SECTION 3.2.9
# generator = G(x) = x32 + x26 + x23 + x22 + x16 + x12 + x11 + x10 + x8 + x7 + x5 + x4 + x2 + x + 1
generator = (1<<32) + (1<<26) + (1<<23) + (1<<22) + (1<<16) + (1<<12) + (1<<11) + (1<<10) + (1<<8) + (1<<7) + (1<<5) + (1<<4) + (1<<2) + (1<<1) + (1<<0)
crc32_func = crc_factory(generator)

def crc32(crc_frame_bits, func=crc32_func):
    # step a, complement first 32 bits of frame
    for i in range(32):
        if crc_frame_bits[i] == '1':
            crc_frame_bits[i] = '0'
        else:
            crc_frame_bits[i] = '1'
    
    # step b, take the n bits in the order the arrived in, from destination mac start to end of frame, as coefficients
    crc_frame_bits = crc_frame_bits[:-32] # just lop off the crc that came with it
    
    # step c, shift bits by 32 and fill with 0, then divide by the generator
    # when using a library for crc calcs, DO NOT SHIFT THE INPUT by 32, since they will automatically do this..
    crc_frame_bin = int(''.join(crc_frame_bits), base=2) << 32
    
    # DO CRC CALCULATION AND GET REMAINDER 
    r = crc32_func(crc_frame_bin)
    t = ~r  # ^ 0xFFFFFFFF # one's-complement the output as required by 802.3
    return t & 0xFFFFFFFF # mask to only keep the first 32 bits

with open('framebits.txt', 'r') as f:
    # read frame-bits 32 from ethernet frame
    # crc included at the end
    # raw bits though as per the mac frame transmission order - not reversed or anything
    s = f.read()

r = crc32(list(s))
known_crc = int(s[-32:], base=2)
assert known_crc == r # crc should be 2523207787 per framebits.txt
print(r)
print(known_crc)
print(bin(r))
print(bin(known_crc))
print(hex(r))
print(hex(known_crc))

2523207787
2523207787
0b10010110011001010001100001101011
0b10010110011001010001100001101011
0x9665186b
0x9665186b
